# Data Cleaning:

To run this notebook, follow the link: https://www.kaggle.com/code/stegosaurus3000/ml-project-data-cleaning/edit/run/250129743

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import folium
import seaborn as sns
import altair as alt

## The Data we want to work with: 
- neighborhoods for geographical information
- listings because it has a lot of informations we need
- reviews since we are doing sentiment analyses on the reviews
- calendar using the median of the prices to fill missing price values 

In [ ]:
neighborhoods = gpd.read_file('/kaggle/input/florence-airbnb-data/neighbourhoods.geojson')
listings = pd.read_csv('/kaggle/input/florence-airbnb-data/listings.csv')
reviews = pd.read_csv('/kaggle/input/florence-airbnb-data/reviews.csv')
calendar = pd.read_csv('/kaggle/input/florence-airbnb-data/calendar.csv', low_memory=False)

## Cleaning listings

In [ ]:
# -> removing cols 
drop_col = [col for col in listings.columns if col.startswith(('minimum', 'maximum', 'number', 'estimated', 'availability', 'host', 'calculated'))]
remove = ['host_id','host_since', 'host_total_listings_count', 'minimum_nights', 'maximum_nights', 'number_of_reviews']
for i in remove:
     drop_col.remove(i) 
    
drop_col_2 = ['neighbourhood_group_cleansed', 'license', 'source', 'scrape_id', 'last_scraped', 'picture_url', 'calendar_updated','first_review',
              'last_review', 'neighbourhood', 'calendar_last_scraped', 'description','neighborhood_overview', 'instant_bookable',
             'reviews_per_month', 'property_type']

drop_col.extend(drop_col_2)

listings = listings.drop(columns=drop_col)

In [ ]:
# cleaning listings -> handling dtypes, NaN, Outliers
listings['price'] = listings['price'].replace('[\$,]', '', regex=True).astype(float)
#listings = listings[(listings['price'].notna()) & (listings['price'] > 0)]
listings['host_since'] = pd.to_datetime(listings['host_since']).dt.year
listings['has_availability'] = (listings['has_availability']
                                    .replace({'t':True})
                                    .mask(listings['has_availability'].isna(), False)
                                    .infer_objects(copy=False)
                                    .astype('boolean'))

In [ ]:
# -> looking at bathroom
'''
bathrooms or bathrooms_text better, difference between private bath and just bath? -> bathrooms_text: more values
change col to numeric and add private or not in additional col?
if nothing is said, meaning neither private or shared, what is it? -> can we assume that private?
regex with the word inbetween number and bath/s
'''
import re

pattern_type = '(?<=\d\s)(.+?)(?=\s*baths?)'
pattern_num = '\d+(?:\.\d+)?'
listings['bathrooms_type'] = listings['bathrooms_text'].apply(
    lambda x : re.search(pattern_type, x).group(1) if isinstance(x, str) and re.search(pattern_type, x) else 'private')
listings['number_of_bathrooms'] = listings['bathrooms_text'].str.extract(
    '[-+]?([0-9]*\.[0-9]+|[0-9]+).').astype(float) # only number of bathrooms
listings['private_bathroom'] = listings.apply(
    lambda row: row['bathrooms_type'] == 'private' and float(row['number_of_bathrooms']) > 0,axis=1)

### Ids/Index to drop because of wrong prices
-> We checked the top 15 outliers in each direction and want to remove them manually because they would add misinformation to our later models

- index_drop_max = [709, 11791, 11966, 11217, 12401, 9819, 11328, 12399, 12259, 7947, 12400]
- id_max = [4125088, 1314427263058807613, 1340352056230924393, 1257165413677461858, 1366555102750755244, 1133220352665651654,1267451639051994556, 1366553621057574692, 1352674337000579304, 917269958869925304, 1366554612907040339]
- index_drop_min = [12223, 5897, 10811, 12204 ]
- id_min = [1351278529533584643, 628055151976628321, 1212870293462675489, 1350659759208899099 ]
- found also one with each 50 bathrooms and bedrooms but the link did not work (index: 9209)

In [ ]:
print(listings.price.describe())
listings_sorted = listings.sort_values(by='price', ascending=False)
#print(listings_sorted.tail(15)[['price', 'id', 'listing_url']])

In [ ]:
listings = listings[listings['price']<2500]
index_drop = [12223, 5897, 10811, 12204, 9209]
listings = listings.drop(index=index_drop)

In [ ]:
print(listings.price.describe())

In [ ]:
listings.info()
listings['review_scores_rating'].describe()

In [ ]:
#Handling the missing review scores
review_cols = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
                          'review_scores_communication', 'review_scores_location', 'review_scores_value']
listings = listings.dropna(subset=review_cols)

In [ ]:
# handling the entries where bed is nan
listings[listings['beds'].isna()][['listing_url','beds', 'bedrooms', 'accommodates']]
valid_beds = listings[listings['beds'].notna()]
bed_medians = valid_beds.groupby(['bedrooms', 'accommodates'])['beds'].median()
def impute_beds(row):
    if pd.isna(row['beds']):
        return bed_medians.get((row['bedrooms'], row['accommodates']), np.nan)
    return row['beds']

listings['beds'] = listings.apply(impute_beds, axis=1)
listings[['bathrooms','bedrooms', 'beds', 'number_of_bathrooms']] = listings[['bathrooms','bedrooms', 'beds', 'number_of_bathrooms']].fillna(0)

### Decide whether we want to have them or some of them in our model

In [ ]:
# -> handling the amenities 
from sklearn.preprocessing import MultiLabelBinarizer
import ast
listings['amenities'] = listings['amenities'].apply(ast.literal_eval) # convert them to lists
listings['amenities'] = listings['amenities'].apply(lambda x: [item.lower() for item in x])

# binary matrix 
mlb = MultiLabelBinarizer()
amenities_encoded = mlb.fit_transform(listings['amenities'])
amenities_df = pd.DataFrame(amenities_encoded, columns=mlb.classes_, index=listings.index)
amenities_df

In [ ]:
luxury_keywords = [ 'pool', 'sauna', 'parking', 'housekeeping', 'air conditioning']


luxury_df = pd.DataFrame(index=amenities_df.index)

for kw in luxury_keywords:
    matched_cols = [col for col in amenities_df if kw in col] # search for keywords
    luxury_df[kw] = amenities_df[matched_cols].sum(axis=1) > 0 # true if keyword is in at least one of the mlb.classes_ 
luxury_df
listings = pd.concat([listings, luxury_df], axis=1)

In [ ]:
listings.info()

In [ ]:
score = pd.read_csv('/kaggle/input/cleaned-df-gesa-csv/score.csv')
score

In [ ]:
listings = listings.merge(score, on='id', how='left')
listings.head()

In [ ]:
listings = listings.drop(columns='amenities')

## Cleaning calendar and using it in listings

In [ ]:
print(calendar.info())
calendar.head()
calendar['date'] = pd.to_datetime(calendar['date'], format='%Y-%m-%d')
calendar['price'] = calendar['price'].replace('[\$,]', '', regex=True).astype(float)
calendar['adjusted_price'] = calendar['adjusted_price'].replace('[\$,]', '', regex=True).astype(float)
calendar['available'] = (calendar['available']
                                      .replace({'t':True, 'f':False})
                                      .infer_objects(copy=False).astype('boolean'))

In [ ]:
# Idea to use calendar: compute for prices and then add to listings and handle missing price values with the mean
calendar_agg = calendar.groupby('listing_id').agg({'price': ['median']}).reset_index()
calendar_agg.columns = ['listing_id', 'price_median']
listings = listings.merge(calendar_agg, left_on='id', right_on='listing_id', how='left')
listings.drop(columns='listing_id', inplace=True)
len(listings[listings[['price', 'price_median']].isna().all(axis=1)]) # is zero 
listings['price'] = listings['price'].fillna(listings['price_median']) # fill therefore the missing prices with the mean
listings[listings['price'].isna()]

In [ ]:
listings.drop(columns='price_median', inplace=True) # drop this column so that our model learn the prices the right way

## Cleaning neigboorhood and incorporating the data in listings

In [ ]:
print(neighborhoods.info())
neighborhoods.drop('neighbourhood_group', axis=1, inplace=True) # all NaN
neighborhoods

In [ ]:
# incorporating the geodata in our listings dataframe 
listings_gdf = gpd.GeoDataFrame(listings, geometry=gpd.points_from_xy(listings.longitude, listings.latitude), crs=neighborhoods.crs)


## Using other geodata
- Finding tourist attractions and important places such as a hospital or central station
- Getting the longitude and latitude coordinates with address using this tool: https://gps-coordinates.org/
- Putting it into Dataframe
- Converting it to Geodataframe
- Combining it with the listings geodataframe
- computing distances in meters

In [ ]:
important_sights = pd.DataFrame({'name': ['Piazza Del Duomo'],
                                 'latitude': ['43.772876'],
                                 'longitude': ['11.255798']})
sights_gdf = gpd.GeoDataFrame(important_sights, geometry=gpd.points_from_xy(important_sights.longitude, important_sights.latitude), crs=neighborhoods.crs)

In [ ]:
# Project to metric CRS so that we can compute the distance in meters later
listings_gdf = listings_gdf.to_crs(epsg=32632)
sights_proj = sights_gdf.to_crs(epsg=32632)
sights_proj

In [ ]:
# have the minimum distance to closest sight
listings_gdf['distance_to_center'] = listings_gdf.geometry.apply(
    lambda geom: sights_proj.distance(geom).min()
)

## Last preparations for our dataframe to be used in our models
- convert or remove object-type columns

In [ ]:
listings = listings_gdf.drop(columns=['geometry']).copy()
listings.drop(['listing_url', 'name'], inplace=True, axis=1)
listings.info()

In [ ]:
listings.drop(columns=['bathrooms', 'bathrooms_text'], inplace=True)

In [ ]:
# convert to numeric values
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
listings_numeric = listings.copy()
columns = listings_numeric.select_dtypes(exclude='number').columns
for col in columns:
    listings_numeric[col] = labelencoder.fit_transform(listings_numeric[col])

listings_numeric.head()

In [ ]:
listings_numeric.info()

In [ ]:
listings_numeric.to_csv('/kaggle/working/airbnb_cleaned_final.csv', index=False) 

## Cleaning reviews

In [ ]:
reviews['date'] = pd.to_datetime(reviews['date'], format='%Y-%m-%d')
reviews = reviews.drop(columns=['reviewer_id', 'reviewer_name', 'id'])
reviews.info()
reviews.head()

# Visualization

In [ ]:
neighbourhood_count = listings.groupby('neighbourhood_cleansed').size().reset_index(name='amount_listings')
alt.Chart(neighbourhood_count, title='Amount of listings in the neighbourhoods').mark_bar().encode(
    x=alt.X('neighbourhood_cleansed', sort='-y').title('Neighbourhood'),
    y=alt.Y('amount_listings').title('Amount of listings')
).properties(width=500,height=500)

In [ ]:
mean_price = listings_numeric['price'].mean()
median_price = listings_numeric['price'].median()

plt.figure(figsize=(10, 6))
sns.histplot(listings_numeric['price'], kde=False, bins=100, color='hotpink')
plt.xlim(0,1200)
plt.title('Price Distribution', size=18, weight='bold')
plt.xlabel('Price in €')
plt.ylabel('Frequency')
plt.axvline(mean_price, color='r', label='mean')
plt.axvline(median_price, color='black', label='median')
plt.legend()
plt.savefig('Price_histogramm.png')
plt.show()

In [ ]:
listings.info()

In [ ]:
num_listing = listings_numeric.select_dtypes('number').drop(columns='price')
col_amount = len(num_listing.columns)
fig, axes = plt.subplots(8,4, figsize=(18,25))

for col in range(col_amount):
    row_idx = col // 4
    col_idx = col % 4
    ax = axes[row_idx, col_idx]
    ax.scatter(num_listing.iloc[:, col], listings_numeric.price, s = 10, alpha = 0.4) # s for marker size
    ax.set_title(num_listing.columns[col], fontsize = 8)

fig.suptitle('AirBnB attributes vs. Price', fontsize=25, weight='bold')
fig.tight_layout(rect=[0.05, 0.05, 1, 0.98])
plt.savefig('Prices_vs_attributes.png')
plt.show()

In [ ]:
col_amount

In [ ]:
prices = listings[listings['price'] < 500] # removing extreme values
fig, ax = plt.subplots(figsize=(12,12))
neighborhoods.plot(ax=ax, color='lightgray', edgecolor='black')
plt.scatter(x=prices['longitude'], y=prices['latitude'], c=prices['price'], s=4, cmap=plt.get_cmap('inferno'))
plt.title('Neighbourhoods of Florence vs. Price', size=18, weight='bold')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.savefig('Neighboorhoods_prices.png');

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
neighborhoods.plot(ax=ax, color='lightgray', edgecolor='black')
sns.scatterplot(x=listings['longitude'], y=listings['latitude'], hue=listings['room_type'], s=8, alpha=0.6, palette='husl')
plt.title('Neighbourhoods of Florence vs. Roomtype')
plt.xlabel('Longitude')
plt.ylabel('Latitude');

In [ ]:
from branca.colormap import linear
from folium.plugins import MarkerCluster, HeatMap
'''
too much data to plot everything, kaggle crashes
'''
Florence = folium.Map(location = [43.77925, 11.24626], zoom_start=13)
colormap = linear.YlGnBu_09.scale(listings['price'].min(), listings['price'].max())
colormap.caption = 'Price (€)'
colormap.add_to(Florence)
marker_cluster = MarkerCluster().add_to(Florence)

# Add markers
for _, row in listings[:3000].iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=colormap(row['price']),
        fill=True,
        fill_opacity=0.7,
        popup=f"€{row['price']}"
    ).add_to(marker_cluster)

# Display the map
Florence

In [ ]:
#a map of Florence
Florence = folium.Map(location = [43.77925, 11.24626], zoom_start=13)

# List comprehension to make a list of lists
heat_data = [[row['latitude'],row['longitude']] for index, row in listings.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(Florence)
Florence

In [ ]:
# Visualizing the neighbourhoods and  given prices of the Airbnbs
# Create base map centered roughly around the data
m = folium.Map(location=[listings['latitude'].mean(), listings['longitude'].mean()], zoom_start=12)


# points for entries 
for _, row in listings.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color='hotpink',
        fill=True,
        fill_opacity=0.4
    ).add_to(m)
    
# add the multipolygon from neigborhood for the shapes
folium.GeoJson(neighborhoods,
              name='neighbourhoods',
              fillColor='gray',
              color='black',
              weight=2,
              fill_opacity=0.2
              ).add_to(m)

# add important sights markers where we computed the distance
for _, row in sights_gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)
m


In [ ]:
numerical_listings_data = listings.select_dtypes('number').corr() # check correlation between columns
#print(numerical_listings_data['price'].values.reshape(-1,1).shape)
sns.heatmap(numerical_listings_data['price'].values.reshape(-1,1), xticklabels=['price'], yticklabels=numerical_listings_data.columns)
plt.title('Correlation Heatmap between AirBnb parameters', size=12, weight='bold')
plt.tight_layout()
plt.savefig('Heatmap.png')
plt.show()